In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
#print long strings without truncating in pandas
pd.options.display.max_colwidth = 500


In [6]:
movie_dataset = pd.read_csv(r"C:\Users\mikel\OneDrive\Research_2020\IMDB_Dataset.csv")
clothing_dataset = pd.read_csv(r"C:\Users\mikel\OneDrive\Research_2020\Womens_Clothing_E-Commerce_Reviews.csv", index_col = 0)
food_dataset = pd.read_csv(r"C:\Users\mikel\OneDrive\Research_2020\Amazon_fine_food_customer_reviews.csv", index_col =[0])

In [ ]:
movie_dataset.head()

In [ ]:
clothing_dataset.head()

In [ ]:
food_dataset.head()

Lets take a look at each of these datasets one at a time 

MOVIE DATASET:

In [ ]:
print("The shape of the movie dataset is: {}".format (movie_dataset.shape))
print('\nSome sample datapoint:\n')

movie_dataset.sample(5)

In [ ]:
#Lets get how balanced our movie dataset is
movie_dataset.sentiment.value_counts(normalize = True)

Our movie dataset is equally balanced. 

In [ ]:
#Do we have any nans?
#which columns have nans
movie_dataset.isnull().sum()

In [ ]:
#Lets convert the sentiments from categorical to numerical features
le = LabelEncoder()
movie_dataset['sentiment'] = le.fit_transform(movie_dataset['sentiment'])
movie_dataset.rename(columns = {'review':'Text', 'sentiment':'ratings'}, inplace=True)
#movie_dataset.rename(columns = {'sentiment':'ratings'}, inplace=True)

movie_dataset.head()

Women E-ommerce clothing dataset

In [ ]:
print("The shape of the women dataset is: {}".format (clothing_dataset.shape))
print('\nSome sample datapoint:\n')

clothing_dataset.sample(5)

In [ ]:
#WE are interested in only 2 columns in this dataset. So we will drop some of the columns

clothing_dataset.columns

In [ ]:
clothing_dataset.info()

In [ ]:
clothing_dataset.drop([col for col in clothing_dataset.columns if col not in ['Review Text', 'Rating']], axis=1, inplace=True)


In [ ]:
clothing_dataset.head(3)

In [ ]:
clothing_dataset.Rating.unique()

In [ ]:
#Do we have any null/empty cell
#which columns have nans
clothing_dataset.isnull().sum()


In [ ]:
clothing_dataset.shape

In [ ]:
#Dropping all the empty/ null cell from the clothing dataset
clothing_dataset = clothing_dataset[~clothing_dataset['Review Text'].isnull()]

clothing_dataset.shape

In [ ]:
#we shall consider all ratings 1, 2 as negative reviews and reviews 4, 5 as positive reviews
#All reviews with Rating = 3 shall be considered neutrl reviews and for simplicity, shall be dropped.

clothing_reviews=clothing_dataset[clothing_dataset.Rating!=3]
clothing_reviews["ratings"]= clothing_dataset["Rating"].apply(lambda x: 1 if x > 3  else 0)
clothing_reviews.head()

In [ ]:
clothing_reviews=clothing_reviews.drop("Rating",axis=1)
clothing_reviews.rename(columns = {'Review Text':'Text'}, inplace=True)

clothing_reviews.head()

In [ ]:
clothing_reviews.ratings.value_counts(normalize = True)
# this dataset is very imbalanced

In [ ]:
sns.set(style = 'darkgrid')
ax = sns.countplot(x = 'ratings', data = clothing_reviews)
ax.set_title('Ratings distribution for women clothing dataset');

Amazon fine food dataset

In [ ]:
food_dataset.head()

In [ ]:
food_dataset.drop([col for col in food_dataset.columns if col not in ['Text','Score']], axis=1, inplace=True)
food_dataset.head()

In [ ]:
food_dataset.Score.unique()

In [ ]:
food_dataset.Score.value_counts(normalize = True)

*Since i want to have a binary classification problem with only positive and negative reviews, I will focus mostly on the reviews with 1, 2 as my negative reviews and the reviews with 4, 5 as my positive reviews.For simplicity,  I will dropp all reviews with 3 as i consider them as neutral reviews. *

In [ ]:
#we shall consider all ratings 1, 2 as negative reviews and reviews 4, 5 as positive reviews

food_reviews=food_dataset[food_dataset.Score!=3]
food_reviews["ratings"]= food_dataset['Score'].apply(lambda x: 1 if x > 3  else 0)
food_reviews=food_reviews.drop("Score",axis=1)

#food_reviews.head()

In [ ]:
food_reviews.ratings.value_counts(normalize = True)
#Also severely imbalanced

this dataset is highly imbalanced. But we will not worrry about it now. We will have to extract an equal number of positive and negative reviews when we are creating our unified dataset.

In [ ]:
#Lets find some empty cell and delete them
food_reviews.isnull().sum()

MULTICLASS DATASET CREATION

Description:

The main aim here was to create a unified dataset(multiclass_customer_review_data), which contains the movie_dataset, the women_dataset and the yelp_dataset into one dataset.

We attributed the

--- label '0' to the negative examples of all the 3 dataset.

--- label '1' for the positive examples from the movie_dataset

--- label '2' for the positive examples form the women_dataset

--- label '3' for the positive examples form the food_dataset

Since we are aiming for a balanced dataset, we are going to creat a balanced dataset for all the 3 datasets we are using, before creating the unified multiclass dataset. 

In [1]:
#Lets get all the positive reviewss together

positive_movies_reviews = movie_dataset[movie_dataset.ratings == 1]

positive_clothing_reviews = clothing_reviews[clothing_reviews.ratings == 1]
positive_clothing_reviews['ratings'].replace(1, 2, inplace = True)

positive_food_reviews = food_reviews[food_reviews.ratings == 1]
positive_food_reviews['ratings'].replace(1, 3, inplace = True)


NameError: name 'movie_dataset' is not defined

In [ ]:
#creating a new column to keep track of the originals in the new datasets

positive_movies_reviews['Originally'] = 'Movies'
positive_clothing_reviews['Originally'] = 'Clothing'
positive_food_reviews['Originally'] = 'Food' 

#positive_clothing_reviews.sample(10)


In [ ]:
#we get all the negative reviews together
negative_movies_reviews = movie_dataset[movie_dataset.ratings == 0]
negative_clothing_reviews = clothing_reviews[clothing_reviews.ratings == 0]
negative_food_reviews = food_reviews[food_reviews.ratings == 0]


In [ ]:
#creating a new column to keep track of their original datasets

negative_movies_reviews['Originally'] = 'Movies'

negative_clothing_reviews['Originally'] = 'Clothing'

negative_food_reviews['Originally'] = 'Food'

#negative_movies_reviews.sample(10)

# Creating our dataset

In [ ]:
positive_dataset = pd.concat([positive_movies_reviews, positive_clothing_reviews, positive_food_reviews], axis = 0, ignore_index = True, sort= False)
#Dataframes were merged/concatenated correctly. We have no duplicate index.
positive_dataset.index[positive_dataset.index.duplicated()].unique()

In [ ]:
positive_dataset.shape

In [ ]:
def count_values(df, feature):
    total=df.loc[:,feature].value_counts()
    percentage=round(df.loc[:,feature].value_counts(normalize = True)*100, 2)
    
    return pd.concat([total, percentage], axis =1, keys = ['Total', 'Percentage'])

In [ ]:
count_values(positive_dataset, 'ratings')

In [ ]:
negative_dataset = pd.concat([negative_movies_reviews, negative_clothing_reviews, negative_food_reviews], axis=0, ignore_index = True, sort = False)
#Checking
negative_dataset.index[negative_dataset.index.duplicated()].unique()

In [ ]:
negative_dataset.shape

The food dataset is very large and imbalanced. In an attempt to creat a balanced dataset, we will use only a sample of positve data from our 3 caterories of datasets so that we have approximately the same amount of positive and negative data when creating our multicalss daatset.

In [ ]:
balanced_pos = positive_dataset[positive_dataset['Originally']=='Food'][:65000].append(positive_dataset[positive_dataset['Originally']=='Movies'], ignore_index = True, sort = False)
balanced_pos = balanced_pos.append(positive_dataset[positive_dataset['Originally']=='Clothing'], ignore_index = True, sort = False)
balanced_pos.shape

In [ ]:
balanced_pos.index[balanced_pos.index.duplicated()].unique()

In [ ]:
#Concatenating the positive and negative exxamples to form our multiclass dataset

multiclass_dataset = pd.concat([negative_dataset, balanced_pos],ignore_index = True, sort = False)
multiclass_dataset = multiclass_dataset.apply(lambda x: x.sample(frac=1).values)#trying to shuffle it a little bit
#multiclass_dataset.head(10)

In [ ]:
multiclass_dataset.ratings.value_counts(normalize = True)

We see how ur unified dataset is distributed

In [ ]:
#Visualisation library
from plotly.offline import iplot
import plotly.graph_objs as go
import chart_studio.plotly as py
import plotly.figure_factory as ff
from plotly.offline import iplot
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')


multiclass_dataset['ratings'].value_counts(normalize=True).iplot(kind='bar',
                                                      yTitle='Percentages', 
                                                      linecolor='black', 
                                                      opacity=0.9,
                                                      color='blue',
                                                      theme='pearl',
                                                      bargap=0.6,
                                                      gridcolor='white',
                                                     
                                                      title='Distribution of Ratings column in the multiclass dataset')

In [ ]:
multiclass_dataset.shape